In [5]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict
from dotenv import dotenv_values
from utils.io_format import output_shape_combine_entities
from utils.process_split import Process_split 


HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
hf_e3c = load_dataset("bio-datasets/e3c")

In [3]:
data_ft = pd.DataFrame(columns=['input', 'output', 'layer'])
splits_dict = {}
splits = ['en.layer1', 'en.layer2', 'en.layer2.validation', 'en.layer3',
          'es.layer1', 'es.layer2', 'es.layer2.validation', 'es.layer3',
          'eu.layer1', 'eu.layer2', 'eu.layer2.validation', 'eu.layer3',
          'it.layer1', 'it.layer2', 'it.layer2.validation', 'it.layer3',
          'fr.layer1', 'fr.layer2', 'fr.layer2.validation', 'fr.layer3']

for split_name in splits:
    processed_split = Process_split.apply(hf_e3c, split_name, enitites_separator_in_output="|||")
    data_ft = Dataset.from_pandas(processed_split)
    splits_dict[split_name] = data_ft

ddict = DatasetDict(splits_dict)
ddict.push_to_hub("ferrazzipietro/e3c_finetuning", token=HF_TOKEN)
    

,input,output,layer,language
0,<s>[INST] Extract the entities contained in th...,"offset: [23, 35] text: hypertension ||| offset...",layer1,en
1,<s>[INST] Extract the entities contained in th...,"offset: [3, 10] text: present ||| offset: [48,...",layer1,en
2,<s>[INST] Extract the entities contained in th...,"offset: [27, 36] text: consulted ||| offset: [...",layer1,en
3,<s>[INST] Extract the entities contained in th...,"offset: [8, 14] text: report ||| offset: [47, ...",layer1,en
4,<s>[INST] Extract the entities contained in th...,"offset: [22, 34] text: hypertensive ||| offset...",layer1,en
...,...,...,...,...
50126,<s>[INST] Extract the entities contained in th...,</s>,layer3,fr
50127,<s>[INST] Extract the entities contained in th...,</s>,layer3,fr
50128,<s>[INST] Extract the entities contained in th...,</s>,layer3,fr
50129,<s>[INST] Extract the entities contained in th...,</s>,layer3,fr


In [ ]:
data_ft = Dataset.from_pandas(data_ft)
data_ft.push_to_hub("ferrazzipietro/e3c_finetuning", token=HF_TOKEN)